<a href="https://colab.research.google.com/github/ManoelLeocardio/Acidentes-Recife-/blob/main/Projeto_ti%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.express as px
!pip install folium pandas geopy
!pip install chart_studio
import chart_studio
import chart_studio.plotly as py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 7.4 MB/s eta 0:00:00


DADOS 2015


In [2]:
df = pd.read_csv('acidentes.csv', sep=';')

In [3]:
df.head(10)

,tipo,situacao,data,hora,bairro,endereco,numero,complemento,natureza,descricao,auto,moto,ciclom,ciclista,pedestre,onibus,caminhao,viatura,outros,vitimas
0,SEM VÍTIMA,CANCELADA,2015-12-03,11:36:00.000,CASA AMARELA,AV NORTE,NaN,SENTIDO SUBURBIO E/F AO SESC,COLISÃO,COLISÃO SEM VITIMAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,SEM VÍTIMA,FINALIZADA,2015-06-01,06:35:00.000,ENCRUZILHADA,EST DE BELEM,NaN,IGREJA BATISTA MEMORIAL DE BELÉM,COLISÃO,SEM VÍTIMAS.,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,SEM VÍTIMA,FINALIZADA,2015-06-01,08:04:00.000,BOA VIAGEM,RUA ERNESTO DE PAULA SANTOS,NaN,SEMAFORO Nº217,COLISÃO,SEM VITIMA,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0
3,SEM VÍTIMA,FINALIZADA,2015-06-01,08:06:00.000,AFOGADOS,AV SUL,NaN,AO LADO DA ESTAÇAO LARGO DA PAZ,COLISÃO,COLISAO S/V,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0
4,SEM VÍTIMA,CANCELADA,2015-06-01,08:13:00.000,MADALENA,RUA BENFICA,NaN,EM FRENTE AO CLUBE INTERNACIONAL,COLISÃO,GM RIBEIRO JA NO LOCAL VT 05,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,SEM VÍTIMA,FINALIZADA,2015-06-01,08:40:00.000,TORRE,RUA REAL DA TORRE,NaN,EM FRENTE AO EXTRA BOM,COLISÃO,COL.S/V,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0
6,SEM VÍTIMA,FINALIZADA,2015-06-01,09:30:00.000,SÃO JOSÉ,C SANTA RITA,NaN,SEMAFORO 571,COLISÃO,COLISAO ENTRE ONIBUS E AUTO S/V,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0
7,COM VÍTIMA,FINALIZADA,2015-06-01,09:10:00.000,PINA,AV HERCULANO BANDEIRA,NaN,EM FRENTE AO SEMÁFORO 137,COLISÃO,COLISÃO ENTRE AUTO E MOTO,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,SEM VÍTIMA,CANCELADA,2015-06-01,09:38:00.000,PINA,AV ANTONIO DE GOES,NaN,E/F A SECRETARIA DE ADM.DO GOVERNO DO EST.,COLISÃO,SEM VÍTIMAS. SENTIDO CIDADE,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0
9,SEM VÍTIMA,CANCELADA,2015-06-01,10:11:00.000,ILHA DO LEITE,AV BEIRA RIO,NaN,EM BAIXO DO PONTILHAO ENTRE O HOPE E HOSPITAL ...,COLISÃO,COLISAO ENTRE AIUTO E MOTO S/V,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
from geopy.geocoders import Nominatim
#pesquisar quais lugares mais tiveram acidentes
count = df['endereco'].value_counts()

top_3 = count[:10]

geolocator = Nominatim(user_agent="my_app")
coordinates = {}
for place in top_3.index:
    location = geolocator.geocode(place)
    coordinates[place] = (location.latitude, location.longitude)

print(coordinates)

{'AV MARECHAL MASCARENHAS DE MORAES': (-8.1217415, -34.9142722), 'AV GOVERNADOR AGAMENON MAGALHAES': (-8.0533935, -34.8966824), 'AV CAXANGA': (-8.0311337, -34.9558536), 'AV RECIFE': (-8.107412, -34.9272316), 'AV NORTE': (-1.3971521, -48.4762264), 'AV ENGENHEIRO DOMINGOS FERREIRA': (-8.0903647, -34.8845811), 'AV ENGENHEIRO ABDIAS DE CARVALHO': (-8.0623704, -34.9319576), 'AV BEBERIBE': (-8.0033039, -34.8974621), 'AV DR JOSE RUFINO': (-8.0901615, -34.954363), 'AV CONSELHEIRO AGUIAR': (-8.1178778, -34.8956419)}


In [5]:
#Aqui irá criar um novo DataFrame, seus lugares com altitudes e latitudes
lugares = {'AV MARECHAL MASCARENHAS DE MORAES': (-8.1081358, -34.9116334), 
       'AV GOVERNADOR AGAMENON MAGALHAES': (-8.0462669, -34.8912236), 
       'AV CAXANGA': (-8.0311337, -34.9558536), 
       'AV RECIFE': (-8.08684, -34.9310375),  
       'AV ENGENHEIRO DOMINGOS FERREIRA': (-8.1299745, -34.9023835), 
       'AV ENGENHEIRO ABDIAS DE CARVALHO': (-8.0655714, -34.9391989), 
       'AV BEBERIBE': (-8.0068872, -34.8909196), 
       'AV DR JOSE RUFINO': (-8.0916495, -34.9491305), 
       'AV CONSELHEIRO AGUIAR': (-8.1290353, -34.9006395)}

lugares = pd.DataFrame(lugares).T.reset_index().rename(columns={'index': 'endereco', 0: 'lat', 1: 'lon'})

print(lugares)

                            endereco       lat        lon
0  AV MARECHAL MASCARENHAS DE MORAES -8.108136 -34.911633
1   AV GOVERNADOR AGAMENON MAGALHAES -8.046267 -34.891224
2                         AV CAXANGA -8.031134 -34.955854
3                          AV RECIFE -8.086840 -34.931038
4    AV ENGENHEIRO DOMINGOS FERREIRA -8.129974 -34.902383
5   AV ENGENHEIRO ABDIAS DE CARVALHO -8.065571 -34.939199
6                        AV BEBERIBE -8.006887 -34.890920
7                  AV DR JOSE RUFINO -8.091650 -34.949131
8              AV CONSELHEIRO AGUIAR -8.129035 -34.900639


In [6]:
fig = px.scatter_mapbox(lugares, lat='lat', lon='lon', hover_name='endereco')

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(
    mapbox={
        'center': {'lon': -34.9, 'lat': -8.05},
        'zoom': 10
    }
)
fig.add_trace(
    go.Scattermapbox(
        name='',
        lat=lugares['lat'],
        lon=lugares['lon'],
        mode='markers',
        marker=dict(size=10, color='red'),
        text=lugares['endereco'],
        hovertemplate='%{text}<extra></extra>',
    )
)


username = 'Logann110'
api_key = 'YNSYZLbvMnmO03biehVX'
fig.update_layout(template='plotly_dark')
#exportando
#url = py.plot(fig, filename='Número de vítimas por natureza', auto_open=False, 
#             username=username, api_key=api_key)
#print(url)
fig.show()

In [7]:
import pandas as pd
import plotly.express as px

# Agrupa os dados por endereço, tipo de acidente e soma a quantidade de vítimas
vitimas_por_endereco_e_tipo_acidente = df.groupby(['natureza'])['vitimas'].sum().reset_index()

vitimas_por_endereco_e_tipo_acidente = vitimas_por_endereco_e_tipo_acidente.query('vitimas > 1')
vitimas_por_endereco_e_tipo_acidente = vitimas_por_endereco_e_tipo_acidente.sort_values(by='vitimas', ascending=False)

# Cria o gráfico interativo
fig = px.bar(vitimas_por_endereco_e_tipo_acidente,
             title='Número de vítimas por natureza',
             x='natureza',
             y='vitimas',
             color='natureza',
             orientation='v',
             labels={'vitimas': 'Quantidade de vítimas', 'natureza': 'Natureza'})
fig.update_layout(template='plotly_dark')
fig.show()

username = 'Logann110'
api_key = 'YNSYZLbvMnmO03biehVX'

#exportando
#url = py.plot(fig, filename='Número de vítimas por natureza', auto_open=False, 
#             username=username, api_key=api_key)
#print(url)

In [8]:
#criando um dataframe com os veiculos e seus valores
df_por_veiculo = pd.DataFrame({
    'veículo': ['carro', 'moto', 'onibus', 'ciclomotor', 'caminhão', 'ciclista'],
    'número_de_acidentes': [
        df['auto'].count(),
        df['moto'].count(),
        df['onibus'].count(),
        df['ciclom'].count(),
        df['caminhao'].count(),
        df['ciclista'].count()
    ]
})

print(df_por_veiculo)


      veículo  número_de_acidentes
0       carro                 6504
1        moto                 1775
2      onibus                 1050
3  ciclomotor                   84
4    caminhão                  797
5    ciclista                   62


In [9]:
# criando um dicionário de cores para cada veículo
colors = {
    'carro': '#1f77b4',
    'moto': '#ff7f0e',
    'onibus': '#2ca02c',
    'ciclomotor': '#d62728',
    'caminhão': '#9467bd',
    'ciclista': '#8c564b'
}

# ordenando
df_por_veiculo = df_por_veiculo.sort_values(by='número_de_acidentes', ascending=False)

data3 = [go.Bar(x=df_por_veiculo['veículo'], y=df_por_veiculo['número_de_acidentes'], marker=dict(color=[colors[v] for v in df_por_veiculo['veículo']]))]

layout3 = go.Layout(title='Número de veículos por acidente',
                   xaxis=dict(title='Veículos', categoryorder='total descending'),
                   yaxis=dict(title='Número de veículos'))


fig3 = go.Figure(data=data3, layout=layout3)
fig3.update_layout(template='plotly_dark')
fig3
#fazer um grafico de pizza

#exportando
#url = py.plot(fig3, filename='Número de veículos por acidente', auto_open=False, 
#             username=username, api_key=api_key)
#print(url)


In [10]:
data3 = [go.Pie(labels=df_por_veiculo['veículo'], values=df_por_veiculo['número_de_acidentes'], 
                marker=dict(colors=[colors[v] for v in df_por_veiculo['veículo']]))]


layout3 = go.Layout(title='Número de veículos por acidente',
                   legend=dict(orientation="h", xanchor="center", x=0.5),
                   )


fig3 = go.Figure(data=data3, layout=layout3)
fig3.update_layout(template='plotly_dark')
fig3

#exportando
#py.sign_in('Logann110', 'fSrz8JO0WVFJyDrjJf3W')
#username = 'Logann110'
#api_key = 'fSrz8JO0WVFJyDrjJf3W'
#url = py.plot(fig3, filename='Número de veículos por acidente', auto_open=False, 
#              username=username, api_key=api_key)
#print(url)


In [11]:
import pandas as pd
import plotly.express as px

#agrupando os dados por endereço, tipo de acidente e soma a quantidade de vítimas
vitimas_por_endereco_e_tipo_acidente = df.groupby(['endereco', 'natureza'])['vitimas'].sum().reset_index()

vitimas_por_endereco_e_tipo_acidente = vitimas_por_endereco_e_tipo_acidente.query('vitimas >= 5 and natureza == "COLISÃO"')
vitimas_por_endereco_e_tipo_acidente = vitimas_por_endereco_e_tipo_acidente.sort_values(by='vitimas', ascending=False)

fig = px.bar(vitimas_por_endereco_e_tipo_acidente,
             title='Número de vítimas por colisão',
             x='vitimas',
             y='endereco',
             color='natureza',
             orientation='h',
             labels={'vitimas': 'Quantidade de vítimas', 'endereco': 'Endereços'})
fig.update_layout(width=800, height=1000) 
fig.update_layout(template='plotly_dark')           
fig 


#exportando
#url = py.plot(fig, filename='Número de vítimas por colisão', auto_open=False, 
#             username=username, api_key=api_key)
#print(url)

In [12]:
import pandas as pd
import plotly.express as px

vitimas_por_endereco_e_tipo_acidente = df.groupby(['endereco', 'natureza'])['vitimas'].sum().reset_index()

vitimas_por_endereco_e_tipo_acidente = vitimas_por_endereco_e_tipo_acidente.query('vitimas >= 2 and (natureza == "ATROPELAMENTO" or natureza == "CHOQUE" or natureza == "CAPOTAMENTO" or natureza == "COLISÃO COM CICLISTA" or natureza == "ACID. DE PERCURSO" or natureza == "ATROPELAMENTO ANIMAL" or natureza == "TOMBAMENTO" or natureza == "ENGAVETAMENTO")')
vitimas_por_endereco_e_tipo_acidente = vitimas_por_endereco_e_tipo_acidente.sort_values(by='vitimas', ascending=False)


fig = px.bar(vitimas_por_endereco_e_tipo_acidente,
             title= 'Número de vítimas por acidente e seus locais',
             x='vitimas',
             y='endereco',
             color='natureza',
             orientation='h',
             labels={'vitimas': 'Quantidade de vítimas', 'endereco': 'Endereços'})
fig.update_layout(width=800, height=800)
fig.update_layout(template='plotly_dark')     
fig 

#exportando
#url = py.plot(fig, filename='Número de vítimas por acidente e seus locais', auto_open=False, 
#             username=username, api_key=api_key)
#print(url)


In [47]:
import pandas as pd
import plotly.graph_objects as go

# Carregar o arquivo CSV em um dataframe do pandas

# Converter a coluna de datas para o formato correto
df['data'] = pd.to_datetime(df['data'], format='%Y-%m')

# Agrupar os acidentes por mês e contar o total de acidentes em cada mês
resumo_por_mes = df['data'].dt.to_period('M').value_counts().sort_index()

# Converter o índice para string
resumo_por_mes.index = resumo_por_mes.index.astype(str)

# Criar o gráfico de linha interativo com marcadores
fig = go.Figure()
fig.add_trace(go.Scatter(x=resumo_por_mes.index, y=resumo_por_mes.values, mode='lines+markers'))

# Definir o layout do gráfico
fig.update_layout(title='Resumo de Acidentes por Mês (2015)',
                  xaxis=dict(title='Mês'),
                  yaxis=dict(title='Total de Acidentes'))
fig.update_layout(template='plotly_dark')


# Exibir o gráfico interativo
fig.show()
#exportando
username = 'Logann110'
api_key = 'MQtTLFrHiR8DkiGF09Lo'
url = py.plot(fig, filename='Resumo de Acidentes por Mês (2015)', auto_open=False, 
             username=username, api_key=api_key)
print(url)


PlotlyRequestError: ignored

In [ ]:
import pandas as pd
import plotly.graph_objects as go


df = pd.read_csv('acidentes.csv', sep=';')
# Extrair apenas as horas da coluna "hora"
df['hora'] = pd.to_datetime(df['hora']).dt.hour

# Realizar a contagem dos acidentes para cada hora
contagem_por_hora = df['hora'].value_counts().sort_index()

# Criar o gráfico de linha interativo
fig = go.Figure(data=go.Scatter(x=contagem_por_hora.index, y=contagem_por_hora.values, mode='lines+markers+text', text=contagem_por_hora.values, textposition='top center', name='Acidentes 2015'))
# Definir o layout do gráfico
fig.update_layout(title='Resumo de Acidentes por Hora Em 2015',
                  xaxis=dict(title='Hora do Dia'),
                  yaxis=dict(title='Total de Acidentes'),
                  legend=dict(title='Horas'))
fig.update_layout(template='plotly_dark')
fig.update_traces(line=dict(color='rgb(31, 119, 180)'), marker=dict(color='rgb(31, 119, 180)'))

# Exibir o gráfico interativo
fig.show()
username = 'Logann110'
api_key = '5JIr6POTKRpt7xMBJwZn'
url = py.plot(fig, filename='Resumo de Acidentes por Hora Em 2015', auto_open=False, 
             username=username, api_key=api_key)
print(url)


In [15]:
df2 = pd.read_csv('acidentes2021.csv', sep=';')


In [16]:
import pandas as pd
import plotly.graph_objects as go

# Carregar os conjuntos de dados em DataFrames
df1 = pd.read_csv('acidentes.csv', sep=';')
df2 = pd.read_csv('acidentes2021.csv', sep=';')

# Extrair apenas as horas da coluna "hora" para 2015
df1['hora'] = pd.to_datetime(df1['hora']).dt.hour

# Extrair apenas as horas da coluna "hora" para 2021
df2['hora'] = pd.to_datetime(df2['hora']).dt.hour

# Realizar a contagem dos acidentes por hora em cada DataFrame
contagem_por_hora1 = df1['hora'].value_counts().sort_index()
contagem_por_hora2 = df2['hora'].value_counts().sort_index()

# Criar o gráfico de linha interativo
fig = go.Figure()

fig.add_trace(go.Scatter(x=contagem_por_hora1.index, y=contagem_por_hora1.values,
                         mode='lines+markers+text', name='Acidentes 2015',
                         text=contagem_por_hora1.values, textposition='top center'))

fig.add_trace(go.Scatter(x=contagem_por_hora2.index, y=contagem_por_hora2.values,
                         mode='lines+markers+text', name='Acidentes 2021',
                         text=contagem_por_hora2.values, textposition='top center'))

fig.update_layout(title='Comparação de Acidentes por Hora Entre 2015 e 2021',
                  xaxis=dict(title='Hora do Dia',tickformat='%H:%M', dtick=2),
                  yaxis=dict(title='Total de Acidentes'),
                  legend=dict(title='Conjunto de Dados'),
                  template='plotly_dark')

fig.update_traces(line=dict(color='rgb(31, 119, 180)'), marker=dict(color='rgb(31, 119, 180)'), selector=dict(name='Acidentes 2015'))
fig.update_traces(line=dict(color='rgb(255, 127, 14)'), marker=dict(color='rgb(255, 127, 14)'), selector=dict(name='Acidentes 2021'))

fig.show()


In [17]:
import pandas as pd
import plotly.graph_objects as go

# Carregar o arquivo CSV em um dataframe do pandas

# Converter a coluna de datas para o formato correto
df2['data'] = pd.to_datetime(df2['data'], format='%Y-%m')

# Agrupar os acidentes por mês e contar o total de acidentes em cada mês
resumo_por_mes = df2['data'].dt.to_period('M').value_counts().sort_index()

# Converter o índice para string
resumo_por_mes.index = resumo_por_mes.index.astype(str)

# Criar o gráfico de linha interativo com marcadores
fig = go.Figure()
fig.add_trace(go.Scatter(x=resumo_por_mes.index, y=resumo_por_mes.values, mode='lines+markers'))

# Definir o layout do gráfico
fig.update_layout(title='Resumo de Acidentes por Mês Em 2021',
                  xaxis=dict(title='Mês'),
                  yaxis=dict(title='Total de Acidentes'))
fig.update_layout(template='plotly_dark')


# Exibir o gráfico interativo
fig.show()


In [18]:
import pandas as pd
import plotly.graph_objects as go

# Carregar os arquivos CSV em dataframes do pandas
df = pd.read_csv('acidentes.csv', sep=';')
df2 = pd.read_csv('acidentes2021.csv', sep=';')

# Converter a coluna de datas para o formato correto nos dataframes
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')
df2['data'] = pd.to_datetime(df2['data'], format='%Y-%m-%d')

# Filtrar os dados de junho a dezembro
df = df[df['data'].dt.month >= 6]
df2 = df2[df2['data'].dt.month >= 6]

# Remover o ano dos dados
df['mes'] = df['data'].dt.month
df2['mes'] = df2['data'].dt.month

# Tradução manual dos nomes dos meses
meses = ['Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# Agrupar os acidentes por mês e contar o total de acidentes em cada mês para cada dataframe
resumo_por_mes_df = df['mes'].value_counts().sort_index()
resumo_por_mes_df2 = df2['mes'].value_counts().sort_index()

# Criar o gráfico de linha interativo com marcadores para cada dataframe
fig = go.Figure()

fig.add_trace(go.Scatter(x=meses, y=resumo_por_mes_df.values, mode='lines+markers', name='Acidentes 2015'))
fig.add_trace(go.Scatter(x=meses, y=resumo_por_mes_df2.values, mode='lines+markers', name='Acidentes 2021'))

# Definir o layout do gráfico
fig.update_layout(title='Comparação de Acidentes por Mês',
                  xaxis=dict(title='Mês'),
                  yaxis=dict(title='Total de Acidentes'),
                  template='plotly_dark')

# Exibir o gráfico interativo
fig.show()


In [19]:
#Aqui irá criar um novo DataFrame, seus lugares com altitudes e latitudes
lugares2 = {'AV MARECHAL MASCARENHAS DE MORAES': (-8.1081358, -34.9116334), 
       'AV GOVERNADOR AGAMENON MAGALHAES': (-8.0462669, -34.8912236), 
       'AV CAXANGA': (-8.0311337, -34.9558536), 
       'AV RECIFE': (-8.08684, -34.9310375),  
       'AV ENGENHEIRO DOMINGOS FERREIRA': (-8.1299745, -34.9023835), 
       'AV ENGENHEIRO ABDIAS DE CARVALHO': (-8.0655714, -34.9391989), 
       'AV BEBERIBE': (-8.0068872, -34.8909196), 
       'AV NORTE MIGUEL ARRAES DE ALENCAR': (-8.0475821, -34.8770101), 
       'AV CONSELHEIRO AGUIAR': (-8.1290353, -34.9006395)}

lugares2 = pd.DataFrame(lugares2).T.reset_index().rename(columns={'index': 'endereco', 0: 'lat', 1: 'lon'})

print(lugares2)



                            endereco       lat        lon
0  AV MARECHAL MASCARENHAS DE MORAES -8.108136 -34.911633
1   AV GOVERNADOR AGAMENON MAGALHAES -8.046267 -34.891224
2                         AV CAXANGA -8.031134 -34.955854
3                          AV RECIFE -8.086840 -34.931038
4    AV ENGENHEIRO DOMINGOS FERREIRA -8.129974 -34.902383
5   AV ENGENHEIRO ABDIAS DE CARVALHO -8.065571 -34.939199
6                        AV BEBERIBE -8.006887 -34.890920
7  AV NORTE MIGUEL ARRAES DE ALENCAR -8.047582 -34.877010
8              AV CONSELHEIRO AGUIAR -8.129035 -34.900639


In [20]:
fig = px.scatter_mapbox(lugares2, lat='lat', lon='lon', hover_name='endereco')

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(
    mapbox={
        'center': {'lon': -34.9, 'lat': -8.05},
        'zoom': 10
    }
)
fig.add_trace(
    go.Scattermapbox(
        name='',
        lat=lugares2['lat'],
        lon=lugares2['lon'],
        mode='markers',
        marker=dict(size=10, color='red'),
        text=lugares2['endereco'],
        hovertemplate='%{text}<extra></extra>',
    )
)


username = 'Logann110'
api_key = 'YNSYZLbvMnmO03biehVX'
fig.update_layout(template='plotly_dark')
fig.show()

In [27]:
df_2018 = pd.read_csv('acidentes_2018.csv', sep=';')
df_2018.rename(columns={'DATA': 'data'}, inplace=True)
df_2018.rename(columns={'natureza_acidente': 'natureza'}, inplace=True)

In [22]:

df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')
df_2018['data'] = pd.to_datetime(df_2018['data'], format='%Y-%m-%d')
df2['data'] = pd.to_datetime(df2['data'], format='%Y-%m-%d')

# Filtrar os dados de junho a dezembro
df = df[df['data'].dt.month >= 6]
df_2018 = df_2018[df_2018['data'].dt.month >= 6]
df2 = df2[df2['data'].dt.month >= 6]

# Remover o ano dos dados
df['mes'] = df['data'].dt.month
df_2018['mes'] = df_2018['data'].dt.month
df2['mes'] = df2['data'].dt.month

# Agrupar os acidentes por mês e contar o total de acidentes em cada mês para cada dataframe
resumo_por_mes_df = df['mes'].value_counts().sort_index()
resumo_por_mes_2018 = df_2018['mes'].value_counts().sort_index()
resumo_por_mes_df2 = df2['mes'].value_counts().sort_index()

# Criar o gráfico de linha interativo com marcadores para cada dataframe
fig = go.Figure()

fig.add_trace(go.Scatter(x=meses, y=resumo_por_mes_df.values, mode='lines+markers', name='Acidentes 2015'))
fig.add_trace(go.Scatter(x=meses, y=resumo_por_mes_2018.values, mode='lines+markers', name='Acidentes 2018'))
fig.add_trace(go.Scatter(x=meses, y=resumo_por_mes_df2.values, mode='lines+markers', name='Acidentes 2021'))

# Definir o layout do gráfico
fig.update_layout(title='Comparação de Acidentes por Mês',
                  xaxis=dict(title='Mês'),
                  yaxis=dict(title='Total de Acidentes'),
                  template='plotly_dark')

# Exibir o gráfico interativo
fig.show()


In [36]:
vitimas_por_natureza_df2 = df2.groupby('tipo')['vitimas'].sum().reset_index()

# Filtrar os casos com mais de 1 vítima
vitimas_por_natureza_df2 = vitimas_por_natureza_df2.query('vitimas > 1')

# Ordenar os dados por quantidade de vítimas em ordem decrescente
vitimas_por_natureza_df2 = vitimas_por_natureza_df2.sort_values(by='vitimas', ascending=False)

# Criar o gráfico de barras interativo
fig = px.bar(vitimas_por_natureza_df2,
             title='Número de vítimas por natureza 2021',
             x='tipo',
             y='vitimas',
             color='tipo',
             labels={'vitimas': 'Quantidade de vítimas', 'tipo': 'tipo'})

fig.update_layout(template='plotly_dark')
fig.show()

In [38]:
vitimas_por_natureza_df2 = df_2018.groupby('tipo')['vitimas'].sum().reset_index()

# Filtrar os casos com mais de 1 vítima
vitimas_por_natureza_df2 = vitimas_por_natureza_df2.query('vitimas > 1')

# Ordenar os dados por quantidade de vítimas em ordem decrescente
vitimas_por_natureza_df2 = vitimas_por_natureza_df2.sort_values(by='vitimas', ascending=False)

# Criar o gráfico de barras interativo
fig = px.bar(vitimas_por_natureza_df2,
             title='Número de vítimas por natureza 2018',
             x='tipo',
             y='vitimas',
             color='tipo',
             labels={'vitimas': 'Quantidade de vítimas', 'tipo': 'tipo'})

fig.update_layout(template='plotly_dark')
fig.show()